# Step 1 | Pulling data from the Bronze
## 1.1 | Setting up the Bronze ADLS connection

In [1]:
from pyspark.sql import SparkSession 
from pyspark.sql.types import * 

# Bronze Storage Account Info | Check Azure Portal if you are unsure #
account_name = 'demostoragebronzeraw' 
container_name = 'AdventureWorksLT' 
relative_path_customer = 'Customer/Landing/' 
relative_path_address = 'Address/Landing/' 

# Complete Path To Files | This could be done with a loop if you have a lot of files #
adls_path_customer = 'abfss://%s@%s.dfs.core.windows.net/%s' % (container_name, account_name, relative_path_customer) 
adls_path_address = 'abfss://%s@%s.dfs.core.windows.net/%s' % (container_name, account_name, relative_path_address) 
print('ADLS Path: ' + adls_path_customer) 
print('ADLS Path: ' + adls_path_address) 

StatementMeta(SparkPool, 6, 1, Finished, Available)

ADLS Path: abfss://demobronzelayer@demostoragebronzeraw.dfs.core.windows.net/AdventureWorksLT/Customer/Landing/
ADLS Path: abfss://demobronzelayer@demostoragebronzeraw.dfs.core.windows.net/AdventureWorksLT/Address/Landing/


## 1.2 | Read files and show Data Frame

In [2]:
# Read the CSV files into a Data Frame
csv_path = adls_path_customer + '*.csv' 
BronzeCustomerDF = spark.read.csv(csv_path, header = 'true') 
csv_path = adls_path_address + '*.csv' 
BronzeAddressDF = spark.read.csv(csv_path, header = 'true') 

#Show the contents of the Data Frame
BronzeCustomerDF.show(5)
BronzeAddressDF.show(5)

StatementMeta(SparkPool, 6, 2, Finished, Available)

+----------+---------+-----+---------+----------+----------+------+--------------------+--------------------+--------------------+------------+--------------------+------------+--------------------+-------------------+----------------+-------------------+
|CustomerID|NameStyle|Title|FirstName|MiddleName|  LastName|Suffix|         CompanyName|         SalesPerson|        EmailAddress|       Phone|        PasswordHash|PasswordSalt|             rowguid|       ModifiedDate|    SourceSystem|      IngestionTime|
+----------+---------+-----+---------+----------+----------+------+--------------------+--------------------+--------------------+------------+--------------------+------------+--------------------+-------------------+----------------+-------------------+
|         1|    false|  Mr.|  Orlando|        N.|       Gee|  null|        A Bike Store|adventure-works\p...|orlando0@adventur...|245-555-0173|L/Rlwxzp4w7RWmEgX...|    1KjXYs4=|3F5AE95E-B87D-4AE...|2005-08-01 00:00:00|AdventureWorks

## 1.3 | Create a temporary SQL view from the Data Frame

In [3]:
# View creation #
BronzeCustomerDF.createOrReplaceTempView( "tmpViewBronzeCustomer" )
BronzeAddressDF.createOrReplaceTempView( "tmpViewBronzeAddress" )

StatementMeta(SparkPool, 6, 3, Finished, Available)

## 1.4 | Using SparkSQL to read from the temporary views

In [4]:
%%sql

SELECT * FROM tmpViewBronzeCustomer LIMIT 10;
SELECT * FROM tmpViewBronzeAddress LIMIT 10;

StatementMeta(, 6, -1, Finished, Available)

<Spark SQL result set with 10 rows and 17 fields>

<Spark SQL result set with 10 rows and 5 fields>

# Step 2 | Getting the data ready for Silver
## 2.1 | Consolidation and Standardization

In [5]:
%%sql
/*
In this step we do
- Consolidation: Join multiple source data points and only keep the fields you need in Silver and ignore the fluff. Use UNIONs to group multiple files from different sources.
- Standardization and Formatting: Clean and format the data so that it can align to one standard schema across multiple sources
*/
SELECT 
   CAST('ADW-' || Customer.CustomerID AS VARCHAR(100)) AS CustomerID, 
   CAST(COALESCE(Customer.FirstName,'') || ' ' || COALESCE(Customer.MiddleName,'') || ' ' || COALESCE(Customer.LastName,'') AS VARCHAR(200)) AS CustomerName, 
   CAST(Customer.Phone AS VARCHAR(20)) AS CustomerPhone, 
   CAST(Customer.EmailAddress AS VARCHAR(100)) AS CustomerEmail,
   CAST(Address.AddressID AS INT)  AS CustomerAddressID,
   CAST('AdventureWorksLT' AS VARCHAR(20)) AS SourceSystem,
   CAST(Customer.ModifiedDate AS TIMESTAMP) AS ModifiedDate 
FROM 
    tmpViewBronzeCustomer Customer
INNER JOIN
    tmpViewBronzeAddress Address
    ON Customer.CustomerID = Address.CustomerID
LIMIT 10

StatementMeta(SparkPool, 6, 6, Finished, Available)

<Spark SQL result set with 10 rows and 7 fields>

## 2.2 | Setting up the Silver ADLS connection

In [6]:
# Silver Storage Account Info | Check Azure Portal if you are unsure #
account_name = 'demostoragesilver' 
container_name = 'sales' 
relative_path = 'CustomerMaster/Delta/'
adls_path = 'abfss://%s@%s.dfs.core.windows.net/%s' % (container_name, account_name, relative_path) 
print('ADLS Path: ' + adls_path) 

StatementMeta(SparkPool, 6, 7, Finished, Available)

ADLS Path: abfss://demosilverlayer@demostoragesilver.dfs.core.windows.net/CustomerMaster/Delta/


## 2.3 | Setting up a Delta table

In [61]:
%%sql
/*Use the ADLS Path from the step above and paste in the LOCATION area below*/
DROP TABLE CustomerMaster;
CREATE TABLE CustomerMaster 
(
    CustomerID  string,
    CustomerName  string,
	CustomerPhone  string,
	CustomerEmail  string,
	CustomerAddressID  int,
	SourceSystem  string,
	ModifiedDate  timestamp
) 
USING DELTA
LOCATION 'abfss://sales@demostoragesilver.dfs.core.windows.net/CustomerMaster/Delta/'
/*Verify that the table has been created by going to the storage location and looking for a 'delta_log' directory*/

StatementMeta(, 5, -1, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

# Step 3 | Loading the Silver Layer
## 3.1 | Creating temporary view to load the Delta table

In [7]:
%%sql
/*Creating a temporary view for all the new and updated rows that need to be loaded into Delta
There will be a seperate logic to figure out the rows that need to be deleted*/
CREATE OR REPLACE TEMP VIEW tmpViewSilverCustomerMaster_InsertUpdate AS
(
  SELECT 
    CAST('ADW-' || Customer.CustomerID AS VARCHAR(100)) AS CustomerID, 
    CAST(COALESCE(Customer.FirstName,'') || ' ' || COALESCE(Customer.MiddleName,'') || ' ' || COALESCE(Customer.LastName,'') AS VARCHAR(200)) AS CustomerName, 
    CAST(Customer.Phone AS VARCHAR(20)) AS CustomerPhone, 
    CAST(Customer.EmailAddress AS VARCHAR(100)) AS CustomerEmail,
    CAST(Address.AddressID AS INT)  AS CustomerAddressID,
    CAST('AdventureWorksLT' AS VARCHAR(20)) AS SourceSystem,
    CAST(Customer.ModifiedDate AS TIMESTAMP) AS ModifiedDate 
  FROM 
      tmpViewBronzeCustomer Customer
  INNER JOIN
      tmpViewBronzeAddress Address
      ON Customer.CustomerID = Address.CustomerID
);


StatementMeta(SparkPool, 6, 8, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

## 3.2 Loading Delta table

In [62]:
%%sql
/* --Sample of how MERGE statements works--

MERGE INTO target t
USING (
  -- Find the latest change for each key based on the timestamp
  SELECT key, latest.newValue as newValue, latest.deleted as deleted FROM (    
    -- Note: For nested structs, max on struct is computed as 
    -- max on first struct field, if equal fall back to second fields, and so on.
    SELECT key, max(struct(time, newValue, deleted)) as latest FROM changes GROUP BY key
  )
) s
ON s.key = t.key
WHEN MATCHED AND s.deleted = true THEN DELETE
WHEN MATCHED THEN UPDATE SET key = s.key, value = s.newValue
WHEN NOT MATCHED AND s.deleted = false THEN INSERT (key, value) VALUES (key, newValue)
*/

MERGE INTO CustomerMaster
USING tmpViewSilverCustomerMaster_InsertUpdate CM_IU
ON CustomerMaster.CustomerID = CM_IU.CustomerID
WHEN MATCHED THEN UPDATE SET *
WHEN NOT MATCHED THEN INSERT *

StatementMeta(SparkPool, 5, 31, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

## 3.3 | Reading Delta table

In [8]:
%%sql

SELECT *
FROM CustomerMaster 
ORDER BY CustomerID
LIMIT 10;

SELECT COUNT(*) AS CNT
FROM CustomerMaster;

StatementMeta(, 6, -1, Finished, Available)

<Spark SQL result set with 10 rows and 7 fields>

<Spark SQL result set with 1 rows and 1 fields>

## 3.4 | Showing the Delta history and time travel

In [9]:
%%sql
DESCRIBE HISTORY CustomerMaster

/*
SELECT *
FROM CustomerMaster 
TIMESTAMP AS OF '2022-12-09T05:23:45Z'
*/

StatementMeta(SparkPool, 6, 11, Finished, Available)

<Spark SQL result set with 2 rows and 15 fields>

In [10]:
CustomerMasterAsOfDF = (spark
    .read
    .format("delta")
    .option("versionAsOf", 1)
    .load("abfss://sales@demostoragesilver.dfs.core.windows.net/CustomerMaster/Delta/")
)

#.option("TimestampAsOf", '2022-12-09T05:23:41Z')

CustomerMasterAsOfDF.orderBy("CustomerID").show(10)

StatementMeta(SparkPool, 6, 12, Finished, Available)

+----------+------------------+-------------+--------------------+-----------------+----------------+-------------------+
|CustomerID|      CustomerName|CustomerPhone|       CustomerEmail|CustomerAddressID|    SourceSystem|       ModifiedDate|
+----------+------------------+-------------+--------------------+-----------------+----------------+-------------------+
| ADW-29485| Catherine R. Abel| 747-555-0171|catherine0@advent...|             1086|AdventureWorksLT|2009-05-16 16:33:33|
| ADW-29486|  Kim  Abercrombie| 334-555-0137|kim2@adventure-wo...|              621|AdventureWorksLT|2009-05-16 16:33:33|
| ADW-29489|  Frances B. Adams| 991-555-0183|frances0@adventur...|             1069|AdventureWorksLT|2009-05-16 16:33:33|
| ADW-29490| Margaret J. Smith| 959-555-0151|margaret0@adventu...|              887|AdventureWorksLT|2009-05-16 16:33:33|
| ADW-29492|        Jay  Adams| 158-555-0142|jay1@adventure-wo...|              618|AdventureWorksLT|2009-05-16 16:33:33|
| ADW-29494|Samuel N. Ag

## 3.X | Bonus: Find out how to deal with deleted records from the source

In [11]:
# Hint
# Research SQL Serverless in Azure Synapse
# Use that a comparison point and compare your temporary view with the Serverless Silver object
# Remove those records that are in the Serverless object but not in the current loads view using the MERGE statement
# Sample code on how to connect to a Serverless SQL object is show below


# Azure SQL servername is in the format "jdbc:sqlserver://<AzureSQLServerName>.database.windows.net:1433"
# Synapse SQL serverless name is in the format "jdbc:sqlserver://<AzureSynapsename>-ondemand.sql.azuresynapse.net:1433" 
#servername = "jdbc:sqlserver://supply-chain-synapse-ondemand.sql.azuresynapse.net:1433"
#dbname = "SilverDataLake"
#url = servername + ";" + "databaseName=" + dbname + ";"
#dbtable = "vwSilverNotebookDemoProducts"
#user= mssparkutils.credentials.getSecret('kv-SupplyChainHack','synapseusername')
#password = mssparkutils.credentials.getSecret('kv-SupplyChainHack','synapsepassword')

#Read from SQL table using MS SQL Connector
#print("read data from SQL server table  ")
#SilverProductsDF = spark.read \
#        .format("com.microsoft.sqlserver.jdbc.spark") \
#        .option("url", url) \
#        .option("dbtable", dbtable) \
#        .option("user", user) \
#        .option("password", password).load()

#SilverProductsDF.orderBy("ProductID").show(10)

#SilverProductsDF.createOrReplaceTempView( "tmpSilverProducts" )

#SELECT * FROM tmpSilverProducts ORDER BY ProductID

StatementMeta(SparkPool, 6, 13, Finished, Cancelled)